<a href="https://colab.research.google.com/github/naivea/DataScienceHandbook/blob/master/Ashrae_3_Copy1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
warnings.filterwarnings('ignore')
from time import time
import datetime
# %matplotlib inline
plt.style.use('seaborn')

from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import RobustScaler, StandardScaler, LabelEncoder, Imputer
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, train_test_split
# from sklearn.linear_model import LinearRegression, Ridge, Lasso
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
# from xgboost import XGBRegressor
import lightgbm as lgb


In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"achenjian","key":"7065721bebc3429b643c28280074da8e"}'}

In [4]:
!mkdir -p ~/.kaggle 

!cp kaggle.json ~/.kaggle/ 

!chmod 600 ~/.kaggle/kaggle.json 
! kaggle --version

Kaggle API 1.5.4


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
! kaggle competitions download -c ashrae-energy-prediction

  0% 0.00/2.53M [00:00<?, ?B/s]
100% 2.53M/2.53M [00:00<00:00, 82.8MB/s]
  0% 0.00/44.5k [00:00<?, ?B/s]
100% 44.5k/44.5k [00:00<00:00, 39.2MB/s]
  0% 0.00/1.27M [00:00<?, ?B/s]
100% 1.27M/1.27M [00:00<00:00, 84.8MB/s]
 99% 118M/120M [00:00<00:00, 156MB/s] 
100% 120M/120M [00:00<00:00, 164MB/s]
 74% 65.0M/88.4M [00:00<00:00, 154MB/s]
100% 88.4M/88.4M [00:00<00:00, 179MB/s]
 90% 150M/167M [00:00<00:00, 179MB/s]
100% 167M/167M [00:00<00:00, 181MB/s]


In [6]:
! unzip /content/test.csv.zip
! unzip /content/sample_submission.csv.zip
! unzip /content/train.csv.zip
! unzip /content/weather_test.csv.zip
! unzip /content/weather_train.csv.zip

Archive:  /content/test.csv.zip
  inflating: test.csv                
Archive:  /content/sample_submission.csv.zip
  inflating: sample_submission.csv   
Archive:  /content/train.csv.zip
  inflating: train.csv               
Archive:  /content/weather_test.csv.zip
  inflating: weather_test.csv        
Archive:  /content/weather_train.csv.zip
  inflating: weather_train.csv       


In [0]:
# path = '../input/ashrae-energy-prediction/'
path = './'
pd.set_option('max_colwidth',200)
pd.set_option('display.width',200)
pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',1000)

In [0]:

# Original code from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin
# Modified to support timestamp type, categorical type
# Modified to add option to use float16

from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df
# def import_data(file):
#     """create a dataframe and optimize its memory usage"""
#     df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
#     df = reduce_mem_usage(df,use_float16=True)
#     return df
def weather_set(weather_df):
     # Find Missing Dates
    time_format = "%Y-%m-%d %H:%M:%S"
    start_date = datetime.datetime.strptime(weather_df['timestamp'].min(),time_format)
    end_date = datetime.datetime.strptime(weather_df['timestamp'].max(),time_format)
    total_hours = int(((end_date - start_date).total_seconds() + 3600) / 3600)
    hours_list = [(end_date - datetime.timedelta(hours=x)).strftime(time_format) for x in range(total_hours)]
    
    missing_hours = []
    for site_id in range(16):
        site_hours = np.array(weather_df[weather_df['site_id'] == site_id]['timestamp'])
        new_rows = pd.DataFrame(np.setdiff1d(hours_list,site_hours),columns=['timestamp'])
        new_rows['site_id'] = site_id
        weather_df = pd.concat([weather_df,new_rows])

        weather_df = weather_df.reset_index(drop=True)           

    # Add new Features
    weather_df["datetime"] = pd.to_datetime(weather_df["timestamp"])
    weather_df["day"] = weather_df["datetime"].dt.day
    weather_df["week"] = weather_df["datetime"].dt.week
    weather_df["month"] = weather_df["datetime"].dt.month
    
    # Reset Index for Fast Update
    weather_df = weather_df.set_index(['site_id','day','month'])

    air_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['air_temperature'].mean(),columns=["air_temperature"])
    weather_df.update(air_temperature_filler,overwrite=False)

    # Step 1
    cloud_coverage_filler = weather_df.groupby(['site_id','day','month'])['cloud_coverage'].mean()
    # Step 2
    cloud_coverage_filler = pd.DataFrame(cloud_coverage_filler.fillna(method='ffill'),columns=["cloud_coverage"])

    weather_df.update(cloud_coverage_filler,overwrite=False)

    due_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['dew_temperature'].mean(),columns=["dew_temperature"])
    weather_df.update(due_temperature_filler,overwrite=False)

    # Step 1
    sea_level_filler = weather_df.groupby(['site_id','day','month'])['sea_level_pressure'].mean()
    # Step 2
    sea_level_filler = pd.DataFrame(sea_level_filler.fillna(method='ffill'),columns=['sea_level_pressure'])

    weather_df.update(sea_level_filler,overwrite=False)

    wind_direction_filler =  pd.DataFrame(weather_df.groupby(['site_id','day','month'])['wind_direction'].mean(),columns=['wind_direction'])
    weather_df.update(wind_direction_filler,overwrite=False)

    wind_speed_filler =  pd.DataFrame(weather_df.groupby(['site_id','day','month'])['wind_speed'].mean(),columns=['wind_speed'])
    weather_df.update(wind_speed_filler,overwrite=False)

    # Step 1
    precip_depth_filler = weather_df.groupby(['site_id','day','month'])['precip_depth_1_hr'].mean()
    # Step 2
    precip_depth_filler = pd.DataFrame(precip_depth_filler.fillna(method='ffill'),columns=['precip_depth_1_hr'])

    weather_df.update(precip_depth_filler,overwrite=False)

    weather_df = weather_df.reset_index()
    weather_df = weather_df.drop(['datetime','day','week','month'],axis=1)
        
    return weather_df

def features_engineering(df):
    
    # Sort by timestamp
    df.sort_values("timestamp")
    df.reset_index(drop=True)
    
    # Add more features
    df["timestamp"] = pd.to_datetime(df["timestamp"],format="%Y-%m-%d %H:%M:%S")
    df["hour"] = df["timestamp"].dt.hour
    df["weekend"] = df["timestamp"].dt.weekday
    df['square_feet'] =  np.log1p(df['square_feet'])
    holidays = ["2016-01-01", "2016-01-18", "2016-02-15", "2016-05-30", "2016-07-04",
                "2016-09-05", "2016-10-10", "2016-11-11", "2016-11-24", "2016-12-26",
                "2017-01-02", "2017-01-16", "2017-02-20", "2017-05-29", "2017-07-04",
                "2017-09-04", "2017-10-09", "2017-11-10", "2017-11-23", "2017-12-25",
                "2018-01-01", "2018-01-15", "2018-02-19", "2018-05-28", "2018-07-04",
                "2018-09-03", "2018-10-08", "2018-11-12", "2018-11-22", "2018-12-25",
                "2019-01-01"]
    df["Holiday"] = (df.timestamp.isin(holidays)).astype(int)
    
    cols=['sea_level_pressure','wind_direction','wind_speed']
    for col in cols:
        df[col].fillna(df[col].mean(), inplace=True)
    
    # Remove Unused Columns
    drop = ["timestamp","floor_count"]
    df = df.drop(drop, axis=1)
    gc.collect()
    
    df['year_built'].fillna(-1, inplace=True)
    
    # Encode Categorical Data
    df['primary_use'].replace({"Healthcare":"Other","Parking":"Other","Warehouse/storage":"Other","Manufacturing/industrial":"Other",
                                "Retail":"Other","Services":"Other","Technology/science":"Other","Food sales and service":"Other",
                                "Utility":"Other","Religious worship":"Other"},inplace=True)
    le = LabelEncoder()
    df["primary_use"] = le.fit_transform(df["primary_use"])
    
    return df


# Start 

In [0]:
train = pd.read_csv(path + "train.csv")
# Remove outliers
train = train[train['building_id'] != 1099]
train = train.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

weather_train = pd.read_csv(path + 'weather_train.csv')
building = pd.read_csv(path + 'building_metadata.csv')

In [10]:
weather = weather_set(weather_train) 
train = reduce_mem_usage(train,use_float16=True)
building = reduce_mem_usage(building,use_float16=True)
weather = reduce_mem_usage(weather,use_float16=True)

Memory usage of dataframe is 757.31 MB
Memory usage after optimization is: 322.24 MB
Decreased by 57.4%
Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 73.8%
Memory usage of dataframe is 9.65 MB
Memory usage after optimization is: 2.66 MB
Decreased by 72.5%


In [11]:
%%time
train = pd.merge(train,building,on='building_id',how='left')
print (train.shape)
train = pd.merge(train,weather_train,on=['site_id','timestamp'],how='left')
print (train.shape)
del weather_train
gc.collect()

(19852422, 9)
(19852422, 16)
CPU times: user 7.48 s, sys: 429 ms, total: 7.91 s
Wall time: 7.92 s


In [0]:
train = features_engineering(train)

In [13]:
train.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,weekend,Holiday
0,105,0,23.303600,1,0,10.832181,-1.0,3.8,NaN,2.4,NaN,1020.9,240.0,3.1,0,4,1
1,106,0,0.374600,1,0,8.589514,-1.0,3.8,NaN,2.4,NaN,1020.9,240.0,3.1,0,4,1
2,106,3,0.000000,1,0,8.589514,-1.0,3.8,NaN,2.4,NaN,1020.9,240.0,3.1,0,4,1
3,107,0,175.184006,1,0,11.487946,2005.0,3.8,NaN,2.4,NaN,1020.9,240.0,3.1,0,4,1
4,108,0,91.265297,1,0,11.309352,1913.0,3.8,NaN,2.4,NaN,1020.9,240.0,3.1,0,4,1


In [0]:
y = np.log1p(train["meter_reading"])
train = train.drop('meter_reading', axis = 1)

In [0]:
%%time
categorical_features = ["building_id", "site_id", "meter", "primary_use", "weekend"]
# params = {
#     "objective": "regression",
#     "boosting": "gbdt",
#     "num_leaves": 1280,
#     "learning_rate": 0.05,
#     "feature_fraction": 0.85,
#     "reg_lambda": 2,
#     "metric": "rmse",
# }
params = {'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'objective': 'regression',
          'max_depth': -1,
          'learning_rate': 0.1,
          "boosting_type": "gbdt",
          "bagging_seed": 13,
          "metric": 'rmse',
          "verbosity": -1,
          'reg_alpha': 0.5,
          'reg_lambda': 0.5,
          'random_state': 2019}

kf = KFold(n_splits=3)
models = []
for train_index,test_index in kf.split(train):
    train_features = train.loc[train_index]
    train_target = y.loc[train_index]
    
    test_features = train.loc[test_index]
    test_target = y.loc[test_index]
    
    d_training = lgb.Dataset(train_features, label=train_target,categorical_feature=categorical_features, free_raw_data=False)
    d_test = lgb.Dataset(test_features, label=test_target,categorical_feature=categorical_features, free_raw_data=False)
    
    model = lgb.train(params, train_set=d_training, num_boost_round=2000, valid_sets=[d_training,d_test], verbose_eval=25, early_stopping_rounds=100)
    models.append(model)
    del train_features, train_target, test_features, test_target, d_training, d_test
    gc.collect()

Training until validation scores don't improve for 100 rounds.
[25]	training's rmse: 1.2739	valid_1's rmse: 1.4043
[50]	training's rmse: 1.0994	valid_1's rmse: 1.23872
[75]	training's rmse: 1.0381	valid_1's rmse: 1.19879
[100]	training's rmse: 0.994524	valid_1's rmse: 1.17718
[125]	training's rmse: 0.974298	valid_1's rmse: 1.16488
[150]	training's rmse: 0.954417	valid_1's rmse: 1.15286
[175]	training's rmse: 0.942138	valid_1's rmse: 1.14754
[200]	training's rmse: 0.931975	valid_1's rmse: 1.14505
[225]	training's rmse: 0.921385	valid_1's rmse: 1.14169
[250]	training's rmse: 0.916694	valid_1's rmse: 1.14031
[275]	training's rmse: 0.911791	valid_1's rmse: 1.13923
[300]	training's rmse: 0.907942	valid_1's rmse: 1.13787
[325]	training's rmse: 0.904348	valid_1's rmse: 1.13707
[350]	training's rmse: 0.900621	valid_1's rmse: 1.13598


In [0]:
del train, y
gc.collect()

In [0]:
plt.style.use('seaborn')
for model in models:
    lgb.plot_importance(model)
    plt.show()

# Test

In [0]:
test = pd.read_csv(path + 'test.csv')
row_ids = test["row_id"]
test.drop("row_id", axis=1, inplace=True)
test = reduce_mem_usage(test)

Memory usage of dataframe is 954.38 MB
Memory usage after optimization is: 199.59 MB
Decreased by 79.1%


In [0]:
test = test.merge(building,left_on='building_id',right_on='building_id',how='left')
del building
gc.collect()

13843

In [0]:
weather_test = pd.read_csv(path + 'weather_test.csv')
weather_test = weather_set(weather_test)
weather_test = reduce_mem_usage(weather_test)

Memory usage of dataframe is 19.25 MB
Memory usage after optimization is: 9.05 MB
Decreased by 53.0%


In [0]:
test = test.merge(weather_test,how='left',on=['timestamp','site_id'])
del weather_test
gc.collect()

0

In [0]:
test = features_engineering(test)

In [0]:
test.head()

,building_id,meter,site_id,primary_use,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,weekend,Holiday
0,0,0,0,0,8.913685,2008.0,17.799999,4.0,11.7,0.282609,1021.400024,100.0,3.6,0,6,0
1,1,0,0,0,7.908755,2004.0,17.799999,4.0,11.7,0.282609,1021.400024,100.0,3.6,0,6,0
2,2,0,0,0,8.589886,1991.0,17.799999,4.0,11.7,0.282609,1021.400024,100.0,3.6,0,6,0
3,3,0,0,0,10.072639,2002.0,17.799999,4.0,11.7,0.282609,1021.400024,100.0,3.6,0,6,0
4,4,0,0,0,11.666573,1975.0,17.799999,4.0,11.7,0.282609,1021.400024,100.0,3.6,0,6,0


# Prediction

In [0]:
%%time

predictions = []
for model in models:
    if  predictions == []:
        predictions = np.expm1(model.predict(test, num_iteration=model.best_iteration)) / len(models)
    else:
        predictions += np.expm1(model.predict(test, num_iteration=model.best_iteration)) / len(models)
    del model
    gc.collect()
del test, models
gc.collect()

CPU times: user 2h 21min 14s, sys: 30.9 s, total: 2h 21min 44s
Wall time: 3min 37s


0

In [0]:
Submission = pd.DataFrame({"row_id": row_ids, "meter_reading": np.clip(predictions, 0, a_max=None)})

Submission.head(20)
Submission.to_csv(path + "submission1.csv",index=None)

In [0]:
Submission.shape

(41697600, 2)